In [1]:
import os
import torch
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModel, pipeline, AutoModelForSequenceClassification
import numpy as np
from datasets import Dataset, load_metric
import pandas as pd


## Getting AUC to run

In [10]:
roc_auc_score = load_metric("roc_auc", "multiclass")
refs = [1, 0, 1, 2, 2, 0]
pred_scores = [[0.3, 0.5, 0.2],
               [0.7, 0.2, 0.1],
               [0.005, 0.99, 0.005],
               [0.2, 0.3, 0.5],
               [0.1, 0.1, 0.8],
               [0.1, 0.7, 0.2]]
results = roc_auc_score.compute(references=refs, prediction_scores=pred_scores, multi_class='ovr')
print(round(results['roc_auc'], 2))

0.85


In [8]:
test_refs = pd.read_csv('labels.csv')
test_refs .drop('Unnamed: 0', axis = 1, inplace = True)

In [2]:
test_preds = pd.read_csv('logits.csv')
test_preds.drop(['Unnamed: 0', 'index'], axis=1 , inplace=True)

In [9]:
nPreds = test_preds.to_numpy()
nRefs = test_refs.to_numpy()

In [10]:
predictions = (nPreds == nPreds.max(axis=1)[:,None]).astype(int)

In [11]:
predictions

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0,

In [13]:
def compute_metrics(eval_pred):
    logits, refs = eval_pred 
    metric = load_metric("roc_auc", "multiclass")
    

    return metric.compute(references=refs, prediction_scores=logits, multi_class='ovr')

In [15]:
# eval_pred = [pred_scores, refs]
# eval_pred = [test_preds, test_refs]
eval_pred = [predictions, nRefs]

In [16]:
compute_metrics(eval_pred)

{'roc_auc': 0.6626208721036307}

In [4]:
print(metric.inputs_description)


Args:
- references (array-like of shape (n_samples,) or (n_samples, n_classes)): Ground truth labels. Expects different input based on use case:
    - binary: expects an array-like of shape (n_samples,)
    - multiclass: expects an array-like of shape (n_samples,)
    - multilabel: expects an array-like of shape (n_samples, n_classes)
- prediction_scores (array-like of shape (n_samples,) or (n_samples, n_classes)): Model predictions. Expects different inputs based on use case:
    - binary: expects an array-like of shape (n_samples,)
    - multiclass: expects an array-like of shape (n_samples, n_classes)
    - multilabel: expects an array-like of shape (n_samples, n_classes)
- average (`str`): Type of average, and is ignored in the binary use case. Defaults to 'macro'. Options are:
    - `'micro'`: Calculates metrics globally by considering each element of the label indicator matrix as a label. Only works with the multilabel use case.
    - `'macro'`: Calculate metrics for each label,

## CNN

In [1]:
import pandas as pd 
from keras.utils import to_categorical
from torch.nn.utils.rnn import pad_sequence


from keras.models import Model
from keras.layers import Input, Embedding
from keras.layers import Bidirectional, LSTM
from keras.layers import Dropout, Dense, Activation

import numpy as np



In [2]:
target = 'label'
input_column = 'cleanTitle'

train_data = pd.read_pickle('../../../Files/Submissions/train/train_split_submission.pickle') 
valid_data = pd.read_pickle('../../../Files/Submissions/train/val_split_submission.pickle')
test_data = pd.read_pickle('../../../Files/Submissions/train/test_split_submission.pickle')

train_data = train_data[[target, input_column]]
valid_data = valid_data[[target, input_column]]
test_data = test_data[[target, input_column]]

data = pd.concat([train_data, valid_data, test_data])


train_instances = train_data[input_column].apply(str).apply(str.split)
train_labels = train_data[target]

# collect known word tokens and tags
wordset, labelset = set(), set()

# collect tags from all data, to prevent unseen labels
labelset.update(set(data[target]))

# get the vocabulary
for words in train_instances:
    wordset.update(set(words))

# map words and tags into ints
PAD = '-PAD-'
UNK = '-UNK-'
word2int = {word: i + 2 for i, word in enumerate(sorted(wordset))}
word2int[PAD] = 0  # special token for padding
word2int[UNK] = 1  # special token for unknown words
 
label2int = {label: i for i, label in enumerate(sorted(labelset))}
# inverted index to translate it back
int2label = {i:label for label, i in label2int.items()}


def convert2ints(instances):
    """
    function to apply the mapping to all words
    """
    result = []
    for words in instances:
        # replace words with int, 1 for unknown words
        word_ints = [word2int.get(word, 1) for word in words]
        result.append(word_ints)
    return result
                          
train_instances_int = convert2ints(train_instances)
train_labels_int = [label2int[label] for label in train_labels]

In [3]:
len(test_data)

1408474

In [3]:
test_instances = test_data[input_column].apply(str).apply(str.split)
test_labels = test_data[target]

test_instances_int = convert2ints(test_instances)
test_labels_int = [label2int[label] for label in test_labels]

# convert dev data
val_instances = valid_data[input_column].apply(str).apply(str.split)
val_labels = valid_data[target]

val_instances_int = convert2ints(val_instances)
val_labels_int = [label2int[label] for label in val_labels]

In [4]:
from keras.utils import to_categorical

train_labels_1hot = to_categorical(train_labels_int, len(label2int))
test_labels_1hot = to_categorical(test_labels_int, len(label2int))
val_labels_1hot = to_categorical(val_labels_int, len(label2int))

train_labels_1hot[0]

array([0., 1., 0.], dtype=float32)

In [5]:
# compute 95th percentile of training sentence lengths
L = sorted(map(len, train_instances))
MAX_LENGTH = L[int(len(L)*0.95)]
print(MAX_LENGTH)

# apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_instances_int = pad_sequences(train_instances_int, padding='post', maxlen=MAX_LENGTH)
test_instances_int = pad_sequences(test_instances_int, padding='post', maxlen=MAX_LENGTH)
val_instances_int = pad_sequences(val_instances_int, padding='post', maxlen=MAX_LENGTH)

print(train_instances[0], len(train_instances[0]))
print(train_instances_int[0], len(train_instances_int[0]))

15
['shill', 'organization'] 2
[22655 17709     0     0     0     0     0     0     0     0     0     0
     0     0     0] 15


In [6]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers import GlobalMaxPooling1D, Dropout
from keras.layers.core import Dense, Activation
import numpy as np

# make it repeatable
np.random.seed(42)

# set parameters of matrices and convolution
embedding_dim = 64
nb_filter = 64
filter_length = 3
hidden_dims = 32
stride_length = 1

inputs = Input((MAX_LENGTH, ), 
               name='word_IDs')
embeddings = Embedding(len(word2int), 
                       embedding_dim, 
                       input_length=MAX_LENGTH)(inputs)
convolution = Conv1D(filters=nb_filter,  # Number of filters to use
                    kernel_size=filter_length, # n-gram range of each filter.
                    padding='same',  #valid: don't go off edge; same: use padding before applying filter
                    activation='relu',
                    strides=stride_length)(embeddings)
convolution2 = Activation(activation='tanh')(convolution)
pooling = GlobalMaxPooling1D()(convolution2)
dropout1 = Dropout(0.2)(pooling)
dense = Dense(hidden_dims, activation='relu')(dropout1)
dropout2 = Dropout(0.2)(dense)
output = Dense(len(label2int), activation='softmax')(dropout2)

model = Model(inputs=[inputs], outputs=[output])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'AUC'])

# get alist of all the layers and their size
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 word_IDs (InputLayer)       [(None, 15)]              0         
                                                                 
 embedding (Embedding)       (None, 15, 64)            1836928   
                                                                 
 conv1d (Conv1D)             (None, 15, 64)            12352     
                                                                 
 activation (Activation)     (None, 15, 64)            0         
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                             

2022-08-15 11:42:16.761199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
# batch size can have a huge effect on performance!
batch_size = 64
epochs = 5

history = model.fit(train_instances_int, train_labels_1hot,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(val_instances_int, val_labels_1hot)
                   )

loss, accuracy, auc = model.evaluate(test_instances_int, test_labels_1hot,
                                batch_size=batch_size,
                                verbose=False)

print("\nTesting Accuracy:  {:.4f}".format(auc))

Epoch 1/5
1079/1079 [==============================] - 24s 22ms/step - loss: 0.0379 - accuracy: 0.9839 - auc: 0.9995 - val_loss: 1.3737 - val_accuracy: 0.8266 - val_auc: 0.9164
Epoch 2/5
1079/1079 [==============================] - 11s 11ms/step - loss: 0.0373 - accuracy: 0.9841 - auc: 0.9995 - val_loss: 1.4615 - val_accuracy: 0.8272 - val_auc: 0.9136
Epoch 3/5
1079/1079 [==============================] - 12s 11ms/step - loss: 0.0381 - accuracy: 0.9837 - auc: 0.9994 - val_loss: 1.4907 - val_accuracy: 0.8253 - val_auc: 0.9125
Epoch 4/5
1079/1079 [==============================] - 13s 12ms/step - loss: 0.0388 - accuracy: 0.9840 - auc: 0.9994 - val_loss: 1.3925 - val_accuracy: 0.8271 - val_auc: 0.9154
Epoch 5/5
1079/1079 [==============================] - 13s 12ms/step - loss: 0.0370 - accuracy: 0.9844 - auc: 0.9995 - val_loss: 1.3994 - val_accuracy: 0.8253 - val_auc: 0.9155

Testing Accuracy:  0.8731


In [29]:
results = model.predict(test_instances_int)

44015/44015 [==============================] - 43s 983us/step


In [28]:
auc

0.9474369287490845

In [30]:
results.tofile('../../../Files/models/cnn_results2.txt', sep=' ')

In [18]:
results.shape

(1408474, 3)

In [46]:
val_instances_int.shape

(15200, 28)

In [44]:
test_instances_int.shape

(1408474, 28)

# Fine Tune BERT

In [55]:
df = pd.read_pickle('../../../Files/Submissions/train/submission_train_sm.pickle')



# tokenizer = AutoTokenizer.from_pretrained("j-hartmann/sentiment-roberta-large-english-3-classes")

# model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/sentiment-roberta-large-english-3-classes")

#, warmup = 600, max_sequence_length=128, training_rate=1e-5,

df['text'] = df['title']
df['label'] = df['label'].fillna(0)
df['label'] = df['label'].astype(int)
df.loc[df["label"] == -1, "label"] = 2
# df['labels'] = df['label']
df = df[['text', 'label']]
# df = df[df['labels'] != '0']
# df = df[df['labels'].notna()]
# df.loc[df["labels"] == -1, "labels"] = 0


model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = Dataset.from_pandas(df, preserve_index=False)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)



tokenized_dataset = dataset.map(tokenize_function, batched=True)

dataset_splitted = tokenized_dataset.shuffle(1337).train_test_split(0.1)

dataset_splitted

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /Users/jakobschlierf/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/r

  0%|          | 0/64 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 57523
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6392
    })
})

In [53]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /Users/jakobschlierf/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tr

In [56]:


for name, param in model.named_parameters():
    if name in ['classifier.weight', 'classifier.bias']:
        param.requires_grad = True
    else:
        param.requires_grad = False

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)




training_args = TrainingArguments(
    load_best_model_at_end=True,
    output_dir = '../../Files/models/bert_base_cased_model/',
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=32, 
    evaluation_strategy='epoch',
    logging_dir='../../Files/logs/', 
    save_strategy = "epoch",
    save_steps=10_000, save_total_limit=2, )



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_splitted['train'],
    eval_dataset=dataset_splitted['test'],
    compute_metrics=compute_metrics,
)


trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 57523
  Num Epochs = 5
  Instantaneous batch size

  0%|          | 0/8990 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [48]:
trainer.train_dataset[0]["label"]